In [7]:
import networkx as nx
import matplotlib.pyplot as plt
import time
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np

In [5]:
def load_graph(graph_file):
    print(f"Loading graph from {graph_file}...")
    start_time = time.time()
    G = nx.read_graphml(graph_file)
    end_time = time.time()
    print(f"Graph loaded in {end_time - start_time:.2f} seconds. It has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    return G

In [13]:
G = load_graph("spotify_AugWeek1.graphml")

Loading graph from spotify_AugWeek1.graphml...
Graph loaded in 3.31 seconds. It has 2114 nodes and 108718 edges.


In [14]:
def girvan_newman_with_tracking(G):
    G_copy = G.copy()
    clusters = {node: [node] for node in G.nodes()}  # Track nodes in clusters
    while G_copy.number_of_edges() > 0:
        centrality = nx.edge_betweenness_centrality(G_copy)
        max_cent = max(centrality, key=centrality.get)
        G_copy.remove_edge(*max_cent)
        components = list(nx.connected_components(G_copy))
        new_clusters = {}
        for comp in components:
            new_cluster = []
            for node in comp:
                new_cluster.extend(clusters.pop(node))
            for node in new_cluster:
                new_clusters[node] = new_cluster
        clusters = new_clusters
        yield clusters

In [15]:
def create_linkage_matrix(G):
    gen = girvan_newman_with_tracking(G)
    cluster_list = list(gen)  # Get list of clustering steps
    n = G.number_of_nodes()
    linkage_matrix = np.zeros((n-1, 4))
    current_cluster_label = -1
    node_to_cluster = {node: node for node in G.nodes()}
    for i, clusters in enumerate(cluster_list[:-1]):  # Last one is all separate, ignore
        merged = set()
        for cluster in clusters.values():
            if len(cluster) > len(merged):
                merged = set(cluster)
        for node in merged:
            old_cluster = node_to_cluster[node]
            linkage_matrix[i, 0] = old_cluster
            linkage_matrix[i, 1] = current_cluster_label
            node_to_cluster[node] = current_cluster_label
        linkage_matrix[i, 2] = i + 1  # Incremental distance
        linkage_matrix[i, 3] = len(merged)  # Cluster size
        current_cluster_label -= 1
    linkage_matrix[:, 0:2] = np.sort(linkage_matrix[:, 0:2], axis=1)  # Sort indices for scipy
    return linkage_matrix

In [16]:
linkage_matrix = create_linkage_matrix(G)

# Plotting the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(linkage_matrix, labels=[str(node) for node in G.nodes()])
plt.title("Dendrogram of Communities Detected via Girvan-Newman Algorithm")
plt.xlabel("Node Index")
plt.ylabel("Level of Division (simulated distance)")
plt.show()

MemoryError: 